In [121]:
import pandas as pd
import re
from datetime import datetime

In [61]:
df = pd.read_csv("orders.csv")
df_ab = pd.read_csv('abandoned.csv')

df.head()

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,...,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References
0,#195763,jasonstuart930@gmail.com,paid,2023-02-15 08:24:52 -0800,fulfilled,2023-02-15 10:46:42 -0800,yes,USD,235.94,22.76,...,NaN,NaN,NaN,NaN,Kentucky,Kentucky,c27962692075603.1,NaN,NaN,c27962692075603.1
1,#195763,jasonstuart930@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,#195764,brian@rossmgt.com,paid,2023-02-15 08:40:31 -0800,fulfilled,2023-02-15 10:54:12 -0800,yes,USD,346.43,35.53,...,NaN,NaN,NaN,NaN,Washington,Washington,c27958684188755.1,NaN,NaN,c27958684188755.1
3,#195764,brian@rossmgt.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,#195764,brian@rossmgt.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# make tables

In [199]:
def Make_Order_Table(df):
    order = df[
        [    'Name','Created at','Financial Status', 'Paid at', 'Fulfillment Status', 
             'Fulfilled at','Accepts Marketing', 'Currency', 'Subtotal', 'Shipping',
             'Taxes','Total', 'Discount Code', 'Discount Amount', 'Shipping Method',
             'Billing Name', 'Billing Address1', 'Billing Address2', 
             'Billing City', 'Billing Zip', 'Billing Province', 'Billing Country',
             'Billing Phone', 'Shipping Name', 'Shipping Address1', 
             'Shipping Address2', 'Shipping City', 'Shipping Zip',
             'Shipping Province', 'Shipping Country', 'Shipping Phone', 'Notes',
             'Payment Method']
    ].copy()

    order['Name'] = order['Name'].str.replace("#", "")
    order['Name'] = order["Name"].drop_duplicates()
    order = order[order["Name"].notna()]
    order.rename(columns =
                    {
                        'Name':'order_id',
                        'Created at':'created_at',
                        'Financial Status':'financial_status',
                        'Paid at':'paid_at',
                        'Fulfillment Status':'fulfillment_status', 
                        'Fulfilled at':'fulfilled_at',
                        'Accepts Marketing':'accepts_marketing',
                        'Currency':'currency',
                        'Subtotal':'subtotal',
                        'Shipping':'shipping',
                        'Taxes':'taxes',
                        'Total':'total',
                        'Discount Code':'discount_code',
                        'Discount Amount':'discount_amount',
                        'Shipping Method':'shipping_method',
                        'Billing Name': 'billing_name',
                        'Billing Address1':'billing_address1',
                        'Billing Address2':'billing_address2',
                        'Billing City':'billing_city',
                        'Billing Zip':'billing_zip',
                        'Billing Province':'billing_province',
                        'Billing Country': 'billing_country',
                        'Billing Phone':'billing_phone',
                        'Shipping Name':'shipping_name',
                        'Shipping Address1' : 'shipping_address1',
                        'Shipping Address2' : 'shipping_address2',
                        'Shipping City':'shipping_city',
                        'Shipping Zip':'shipping_zip',
                        'Shipping Province':'shipping_province',
                        'Shipping Country':'shipping_country',
                        'Shipping Phone':'shipping_phone', 
                        'Notes':'notes',
                        'Payment Method':'payment_method',    
                    }, 
                    
                    inplace = True
                   )
    order.reset_index(drop = True, inplace = True)

    # Convert to datetime object and remove timezone info
    order['created_at'] = pd.to_datetime(order['created_at']).dt.tz_localize(None)
    order['paid_at'] = pd.to_datetime(order['paid_at']).dt.tz_localize(None)
    order['fulfilled_at'] = pd.to_datetime(order['paid_at']).dt.tz_localize(None)
    
    # Format as string in desired format
    order['created_at'] = order['created_at'].dt.strftime('%Y-%m-%d %H:%M:%S')
    order['paid_at'] = order['paid_at'].dt.strftime('%Y-%m-%d %H:%M:%S')
    order['fulfilled_at'] = order['fulfilled_at'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # Fill NaN values
    order['created_at'] = order['created_at'].fillna('2020-01-01 00:00:00')
    order['paid_at'] = order['paid_at'].fillna('2020-01-01 00:00:00')
    order['fulfilled_at'] = order['fulfilled_at'].fillna('2020-01-01 00:00:00')
    order['discount_code'] = order['discount_code'].fillna(0)
#     order['notes'] = order['notes'].fillna('null')
#     order['payment_method'] = order['payment_method'].fillna('null')
#     order['shipping_address2'] = order['shipping_address2'].fillna('null')
    
    return order

def Make_Customer_Table(df):
    customer = df[
        ['Email', 'Billing Name', 'Billing Province']
    ].copy()
    
    customer['Email'] = customer["Email"].drop_duplicates()
    customer = customer[customer["Email"].notna()]
    customer.rename(columns = {
                            'Email':'email',
                            'Billing Name':'full_name',
                            'Billing Province':'location'
                    },                    
                    inplace=True
                   )
    
    customer['full_name'] = customer['full_name'].fillna('0')
    customer.reset_index(drop=True, inplace=True)   
    return customer

def Make_Product_Table(df):
    product = df[
        ['Lineitem sku', 'Lineitem name', 'Lineitem price',
         'Lineitem compare at price','Lineitem requires shipping',
         'Lineitem taxable','Lineitem fulfillment status']
    ].copy()
    
    product['Lineitem name'] = product["Lineitem name"].drop_duplicates()
    product = product[product["Lineitem name"].notna()]
    product['Lineitem price'] = product["Lineitem price"].drop_duplicates()
    product = product[product["Lineitem price"].notna()]
    product.reset_index(drop=True, inplace=True)
    return product

def Make_Order_Item_Table(df):
    order_item = df[
        ['Name', 'Lineitem sku', 'Lineitem quantity', 'Lineitem fulfillment status']
    ].copy()
    order_item['Name'] = order_item['Name'].str.replace("#", "")
    order_item.reset_index(drop=True, inplace=True)
    return order_item

# abandoned_order = Make_Order_Table(df_ab)
# abandoned_order.drop(['Financial Status', 'Paid at', 'Fulfillment Status'], axis=1)

# abandoned_order_item = Make_Order_Item_Table(df_ab)

order_item = Make_Order_Item_Table(df)
product = Make_Product_Table(df)
customer = Make_Customer_Table(df)
order = Make_Order_Table(df)

In [200]:
order

,order_id,created_at,financial_status,paid_at,fulfillment_status,fulfilled_at,accepts_marketing,currency,subtotal,shipping,...,shipping_name,shipping_address1,shipping_address2,shipping_city,shipping_zip,shipping_province,shipping_country,shipping_phone,notes,payment_method
0,195763,2023-02-15 08:24:51,paid,2023-02-15 08:24:52,fulfilled,2023-02-15 08:24:52,yes,USD,235.94,22.76,...,Jason Stuart,2739 Howell Drennen Road,NaN,Mount Sterling,'40353,KY,US,8594045356,NaN,Shopify Payments
1,195764,2023-02-15 08:40:30,paid,2023-02-15 08:40:31,fulfilled,2023-02-15 08:40:31,yes,USD,346.43,35.53,...,Brian Enstad,445 EASY ST,NaN,GOLDENDALE,98620-9037,WA,US,5092611546,Altomino Handmade Butcher Knife - Engraving te...,Shopify Payments
2,195765,2023-02-15 08:43:37,paid,2023-02-15 08:43:38,fulfilled,2023-02-15 08:43:38,yes,USD,304.43,30.96,...,Tamara Ferrigno,10887 Point Rider St,NaN,Las Vegas,'89183,NV,US,7025051050,NaN,Shopify Payments
3,195766,2023-02-15 08:49:19,paid,2023-02-15 08:49:20,fulfilled,2023-02-15 08:49:20,yes,USD,427.84,35.53,...,Michael Wheelerloo,"139 UNION ST, APT C",NaN,ASHLAND,01721-1771,MA,US,5082723832,NaN,Shopify Payments
4,195767,2023-02-15 09:17:21,paid,2023-02-15 09:17:23,fulfilled,2023-02-15 09:17:23,yes,USD,199.94,22.76,...,Joyce Thurler,1413 FOSTER AVE,NaN,JANESVILLE,53545-1082,WI,US,(608) 289-2290,NaN,Shopify Payments
5,195768,2023-02-15 10:06:15,paid,2023-02-15 10:06:17,fulfilled,2023-02-15 10:06:17,yes,USD,256.94,30.96,...,James Supple,303 MAIN ST,NaN,FRANKLIN,70538-6120,LA,US,(337) 578-2734,NaN,Shopify Payments
6,195769,2023-02-15 10:18:00,paid,2023-02-15 10:18:01,fulfilled,2023-02-15 10:18:01,yes,USD,143.46,20.07,...,Blaine Campbell,8750 HIGHWAY 77,NaN,MANSFIELD,38236-3225,TN,US,(731) 415-9151,NaN,Shopify Payments
7,195770,2023-02-15 11:08:25,paid,2023-02-15 11:08:26,fulfilled,2023-02-15 11:08:26,yes,USD,69.97,16.73,...,Michael Lucas,1704 Broadway,NaN,El Cajon,'92021,CA,US,(858) 213-7861,NaN,Shopify Payments
8,195771,2023-02-15 11:22:56,paid,2023-02-15 11:22:57,fulfilled,2023-02-15 11:22:57,yes,USD,230.44,28.48,...,Myrlene M Stimphil,13050 40 Lane N West Pal West Palm Beach FL 33411,"West Palm Beach, FL 33411",Royal Palm Beach,'33411,FL,US,5616921828,NaN,PayPal Express Checkout
9,195772,2023-02-15 11:36:59,paid,2023-02-15 12:42:29,fulfilled,2023-02-15 12:42:29,yes,USD,161.33,20.07,...,Steve Lofgren,5615 220th Street North,NaN,Forest Lake,'55025,MN,US,'+1 952-261-6119,NaN,Shopify Payments


In [198]:
# order.to_csv('orders_table.csv',index = False)

In [218]:
# customer.fillna(0,inplace = True)

# Transfer to DB

In [211]:
import mysql.connector as msql
from mysql.connector import Error

try:
    conn = msql.connect(user='root', password='arm2002',
                              host='127.0.0.1', database='coolina',
                              auth_plugin='mysql_native_password')
    if conn.is_connected():
        cursor = conn.cursor()
#         cursor.execute("CREATE DATABASE irisDB")
        print("irisDB database is created")

except Error as e:
    print("Error while connecting to MySQL", e)

irisDB database is created


In [221]:
try:
    conn = msql.connect(user='root', password='arm2002',
                        host='127.0.0.1', database='coolina',
                        auth_plugin='mysql_native_password')
    
    
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

        for i,row in customer.iterrows():
            sql = "INSERT IGNORE INTO coolina.customer VALUES (%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we must commit to save our changes
            conn.commit()
except Error as e:
    print("Error while connecting to MySQL", e)

You're connected to database:  ('coolina',)
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
